# Neural Network

Neural Network 近十年最有突破的技术，虽然不是万能的，但是在解决很多问题上，已经表现出了划时代的意义。Neural Network 由很多层 layer 组成，每一层 layer 又由很多个 cell 组成。一个 cell 可以是一个简单的 Logistic Regression Model 也可以是 Linear Regression Model，还可以是其他复杂的模型。最近很多年大家在做的事情主要就是改变 layer 的连接结构，改变 cell 所采用的模型。